# LangChain原理

本节介绍对目前最主要的两种使用LLM构建app的方式进行介绍。详细分析了LangChain和Semantic的技术原理和主要组件，并介绍和两个框架的基本使用方法，也对两种框架进行了简单的对比分析。

通过本节学习，你将：

- 掌握以LLM核心的app开发思想
- 掌握LangChain的基本概念、工作流程以及核心组件的使用

## 一、LangChain技术原理

### 什么是LangChain？

>LangChain is a ***framework*** for developing applications powered by ***language models***. It enables applications that:
>
>- **Are context-aware**: connect a language model to sources of context (prompt instructions, few shot examples, content to ground its response in, etc.)
>
>- **Reason**: rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)
>
><p align="right">--<a>https://python.langchain.com/docs/get_started/introduction</a></p>

LangChain是一种用来帮助开发人员使用语言模型构建端到端应用程序的框架，包括一套完整的工具、组件和接口。使用LangChain可以简化创建以LLM和聊天模型为基础的应用程序的过程。它的实现是基于2022年10月发表的论文《ReAct：Synergizing Reasoning And Acting In Language Models》，俗称为ReAct论文。论文提出了一种prompt技术，使得大模型能够先通过推理（reason）然后行动(act)。这里的“推理”即使用chain-of-thought让模型自己生成思考过程，而“行动”即能够通过使用预定义工具集（例如能够搜索互联网）来“行动”。这个思考+行动的组合被证明极大地提高了输出文本的质量，并使得大型语言模型能够回答处理一些更复杂的问题。



#### ReAct实现原理
ReAct通过一段个prompt模板，来组织输入，提示大模型先进行推理，而后再进行行动。prompt模板可以按以下方式进行组织：
```json
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

```
其中，prompt需要输入三个参数：
* `tools`: 包含每个工具的描述和参数。
* `tool_names`: 包含所有工具名称。
* `agent_scratchpad`: 包含以前的代理操作和工具输出作为字符串。


#### Function Calling & Tool Calling

OpenAI的Function calling
```json

// 定义需要提供给模型（如GPT）的工具（函数）列表。
// 模型会根据对话上下文，智能判断是否需要调用以及调用哪个工具。
tools=[
    {
      // 指定工具的类型。这里是 "function"，表示这是一个可供调用的函数。
      "type": "function",
      // 开始定义这个函数的详细信息
      "function": {
        // 函数的唯一名称。模型会在决定调用时返回这个名称。
        // 后端代码根据这个名称来执行对应的真实函数。
        "name": "get_current_temperature",
        // 对函数功能的清晰描述。
        // 这是最重要的部分！模型根据这个描述来判断用户的请求是否需要用这个函数来处理。
        // 描述应简洁准确地说明函数的作用。
        "description": "Get the current temperature for a specific location",
        // 定义这个函数所需要的参数列表及其规范。
        "parameters": {
          // 参数的整体类型，通常是 "object"（对象），因为它包含多个属性。
          "type": "object",
          // 在 "properties" 对象中，详细定义这个函数需要的每一个参数。
          "properties": {
            // 定义第一个参数：location
            "location": {
              // 参数的数据类型，这里是字符串（string）
              "type": "string",
              // 对该参数的描述。模型利用这个描述来理解需要从用户话语中提取什么信息。
              // 例如：用户说“旧金山天气怎么样？”，模型会根据这个描述提取出 "San Francisco, CA"。
              "description": "The city and state, e.g., San Francisco, CA"
            },
            // 定义第二个参数：unit
            "unit": {
              "type": "string",
              // "enum" 表示这个参数的值只能从给定的枚举列表中选择。
              // 模型会从这两个值中二选一，而不是自己生成一个单位。
              "enum": ["Celsius", "Fahrenheit"],
              // 描述告诉模型如何推断应该使用哪个单位。
              // 例如，根据用户所在地（如中国用摄氏，美国用华氏）或用户的提问方式来推断。
              "description": "The temperature unit to use. Infer this from the user's location."
            }
          },
          // "required" 数组列出了函数执行所必需的参数名称。
          // 如果模型无法从用户话语中推断出某个必需参数，它可能会继续追问用户。
          "required": ["location", "unit"]
        }
      }
    }
    // 可以继续在 tools 数组中添加更多的工具（函数）定义...
  ]

``` 
You have access to the following tools:
{function_to_json(get_weather)}
{function_to_json(calculate_mortgage_payment)}
{function_to_json(get_directions)}
{function_to_json(get_article_details)}

You must follow these instructions:
Always select one or more of the above tools based on the user query
If a tool is found, you must respond in the JSON format matching the following schema:
{{
   "tools": {{
        "tool": "<name of the selected tool>",
        "tool_input": <parameters for the selected tool, matching the tool's JSON schema
   }}
}}
If there are multiple tools required, make sure a list of tools are returned in a JSON array.
If there is no tool that match the user request, you will respond with empty json.
Do not add any additional Notes or Explanations

User Query

当我们在 User Query 位置输入下面的话，就会返回一个 json 数据，tools 数组，数组包含一个或者多个 tool 。

```
User Query: What's the weather in London, UK?

Response: { "tools": [ { "tool": "get_weather", "tool_input": { "location": "London, UK" } } ] }

有了这样的 LLM 返回的 json 结构化的数据，接下来要做的工作就是根据 LLM 的选择，来调用执行选择 function 来完成任务，或者是将 function 返回数据再次结合 prompt 返回给 LLM。


接下来我们就来看一看如何从 function 提取一个函数的信息，这些函数信息作为函数的描述，LLM 会根据用户 Query 信息首先选择一个或者多个函数来解决或者回答用户的 Query。

<div class="alert alert-warning">
    
**每次运行前，请先安装以依赖**
</div>

In [1]:
# !pip install -r requirements.txt

In [86]:
# !pip list

In [87]:
messages=[{"role": "system", "content": '''You have access to the following tools:
{tools}
You can select one of the above tools or just response user's content and respond with only a JSON object matching the following schema:
{{
  "tool": <name of the selected tool>,
  "tool_input": <parameters for the selected tool, matching the tool's JSON schema>,
  "message": <direct response users content>
}}'''}]

In [2]:
# 导入必要的库：inspect用于函数检查，json用于数据序列化，typing用于类型提示
import inspect
import json
from typing import get_type_hints

def get_type_name(t):
    """获取类型名称的辅助函数"""
    name= str(t)
    # 如果是复合类型（如list或dict），直接返回字符串表示
    if "list" in name or "dict" in name:
        return name
    else:
        # 否则返回类型的名称
        return t.__name__

def function_to_json(func):
    """
    将Python函数转换为JSON格式的工具描述
    这个函数用于自动生成LLM可以理解的工具描述格式
    """
    # 获取函数的签名信息（参数列表等）
    signature = inspect.signature(func)
    # 获取函数的类型提示信息
    type_hints = get_type_hints(func)

    # 打印函数的文档字符串和类型提示（用于调试）
    print(func.__doc__)
    print(type_hints)

    # 构建符合OpenAI Function Calling格式的函数信息字典
    function_info = {
        "name": func.__name__,  # 函数名称
        "description": func.__doc__,  # 函数描述（来自docstring）
        "parameters": {"type": "object", "properties": {}},  # 参数定义
        "returns": type_hints.get("return", type("void")).__name__,  # 返回值类型
    }

    # 遍历函数的所有参数，提取参数类型信息
    for name, _ in signature.parameters.items():
        param_type = get_type_name(type_hints.get(name, type(None)))
        function_info["parameters"]["properties"][name] = {"type": param_type}

    # 将函数信息转换为格式化的JSON字符串并返回
    return json.dumps(function_info, indent=2)

In [3]:
def get_current_weather(location: str, unit: str) -> str:
    """
    Fetc
    hes the current weather for a given location and unit.
    
    Args:
    location (str): The name of the location.
    unit (str): The unit of temperature (e.g., Celsius, Fahrenheit).
    
    Returns:
    str: A string describing the current weather and temperature.
    """
    return f"It's 20 {unit} in {location}"


In [90]:
function_dict = {
    "get_current_weather": get_current_weather,
}

In [4]:
func_json = function_to_json(get_current_weather)


    Fetc
    hes the current weather for a given location and unit.
    
    Args:
    location (str): The name of the location.
    unit (str): The unit of temperature (e.g., Celsius, Fahrenheit).
    
    Returns:
    str: A string describing the current weather and temperature.
    
{'location': <class 'str'>, 'unit': <class 'str'>, 'return': <class 'str'>}


In [5]:
print(func_json)

{
  "name": "get_current_weather",
  "description": "\n    Fetc\n    hes the current weather for a given location and unit.\n    \n    Args:\n    location (str): The name of the location.\n    unit (str): The unit of temperature (e.g., Celsius, Fahrenheit).\n    \n    Returns:\n    str: A string describing the current weather and temperature.\n    ",
  "parameters": {
    "type": "object",
    "properties": {
      "location": {
        "type": "str"
      },
      "unit": {
        "type": "str"
      }
    }
  },
  "returns": "str"
}


<img src="images/function calling.png" alt="LangChain Framework" width="60%"><br>

In [6]:
# 导入dotenv模块
import dotenv
import os

# 加载.env文件中的环境变量
dotenv.load_dotenv("../.env")

True

In [94]:
# ! pip list
# ! pip install -U openai -i https://pypi.tuna.tsinghua.edu.cn/simple

In [7]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# 使用qwen-max模型进行聊天
qwen = ChatOpenAI(
    model="qwen-max",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

In [8]:
qwen.invoke("你好")

AIMessage(content='你好！有什么可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 9, 'total_tokens': 16, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'id': 'chatcmpl-f0d608c9-031c-4aa7-9cc1-be2392905721', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--e34f3179-95be-4739-8433-2712249b9473-0', usage_metadata={'input_tokens': 9, 'output_tokens': 7, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [10]:
# 从 langchain_core.tools 模块导入 tool 装饰器
from langchain_core.tools import tool

# 使用 @tool 装饰器定义乘法工具函数
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""  # 将两个整数相乘
    return first_int * second_int
 
# 使用 @tool 装饰器定义加法工具函数
@tool
def add(first_int: int, second_int: int) -> int:
    """add two integers together."""  # 将两个整数相加
    return first_int + second_int
    
# 使用 @tool 装饰器定义天气查询工具函数
@tool
def get_current_weather(location: str, unit: str) -> str:
    """
    Fetches the current weather for a given location and unit.
    
    Args:
    location (str): The name of the location. 
    unit (str): The unit of temperature (e.g., Celsius, Fahrenheit). 
    
    Returns:
    str: A string describing the current weather and temperature. 
    """
    return f"It's 20 {unit} in {location}"

# 将所有定义的工具函数收集到一个列表中，供 LangChain 使用
tools = [multiply, add, get_current_weather]

In [11]:
print("Name: ",get_current_weather.name)
print("\nDescription: ",get_current_weather.description)
print("\nArgs: ",get_current_weather.args)

Name:  get_current_weather

Description:  Fetches the current weather for a given location and unit.

Args:
location (str): The name of the location. 
unit (str): The unit of temperature (e.g., Celsius, Fahrenheit). 

Returns:
str: A string describing the current weather and temperature.

Args:  {'location': {'title': 'Location', 'type': 'string'}, 'unit': {'title': 'Unit', 'type': 'string'}}


In [12]:
from langchain.tools.render import render_text_description
 
# 构建工具条件和调用描述
rendered_tools = render_text_description([multiply, add,get_current_weather])
print("rendered_tools = ", rendered_tools)

rendered_tools =  multiply(first_int: int, second_int: int) -> int - Multiply two integers together.
add(first_int: int, second_int: int) -> int - add two integers together.
get_current_weather(location: str, unit: str) -> str - Fetches the current weather for a given location and unit.

Args:
location (str): The name of the location. 
unit (str): The unit of temperature (e.g., Celsius, Fahrenheit). 

Returns:
str: A string describing the current weather and temperature.


In [14]:
# 构建工具选择器的系统人设
system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:
{rendered_tools}
# 构建输入的模板
Given the user input, You can select one of the above tools return the name and input of the tool to use or just response user's content . Return your response as a JSON blob:
"tool": <name of the selected tool>,
"tool_input": <parameters for the selected tool, matching the tool's JSON schema>,
"message": <direct response users content>
"""
print(system_prompt)

You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:
multiply(first_int: int, second_int: int) -> int - Multiply two integers together.
add(first_int: int, second_int: int) -> int - add two integers together.
get_current_weather(location: str, unit: str) -> str - Fetches the current weather for a given location and unit.

Args:
location (str): The name of the location. 
unit (str): The unit of temperature (e.g., Celsius, Fahrenheit). 

Returns:
str: A string describing the current weather and temperature.
# 构建输入的模板
Given the user input, You can select one of the above tools return the name and input of the tool to use or just response user's content . Return your response as a JSON blob:
"tool": <name of the selected tool>,
"tool_input": <parameters for the selected tool, matching the tool's JSON schema>,
"message": <direct response users content>



In [15]:
from langchain_core.prompts import ChatPromptTemplate

# print(system_prompt)
prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)


In [16]:
print(prompt)

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\nmultiply(first_int: int, second_int: int) -> int - Multiply two integers together.\nadd(first_int: int, second_int: int) -> int - add two integers together.\nget_current_weather(location: str, unit: str) -> str - Fetches the current weather for a given location and unit.\n\nArgs:\nlocation (str): The name of the location. \nunit (str): The unit of temperature (e.g., Celsius, Fahrenheit). \n\nReturns:\nstr: A string describing the current weather and temperature.\n# 构建输入的模板\nGiven the user input, You can select one of the above tools return the name and input of the tool to use or just response user\'s content . Return your response as a JSON blob:\n"tool": <name of the selected too

In [17]:
# print(prompt)
chain = prompt | qwen
print(chain.invoke({"input": "993+7等于几"}))

content='{\n  "tool": "add",\n  "tool_input": {\n    "first_int": 993,\n    "second_int": 7\n  }\n}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 234, 'total_tokens': 269, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'id': 'chatcmpl-b74359bc-97cf-473d-a2e9-b6da9cd587bc', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--6f1361e3-4a3a-46ec-a308-01ebd4a177cf-0' usage_metadata={'input_tokens': 234, 'output_tokens': 35, 'total_tokens': 269, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


In [18]:
from langchain_core.output_parsers.json import JsonOutputParser
chain = prompt | qwen | JsonOutputParser()
res = chain.invoke({"input": "993+7等于几"})

In [19]:
res

{'tool': 'add', 'tool_input': {'first_int': 993, 'second_int': 7}}

In [33]:
res = JsonOutputParser().parse('''{\n  "tool": "get_current_weather",\n  "tool_input": {\n    "location": "北京",\n    "unit": "华氏度"\n  },\n  "message": ""\n}
''')

In [34]:
res

{'tool': 'get_current_weather',
 'tool_input': {'location': '北京', 'unit': '华氏度'},
 'message': ''}

In [35]:
# 创建工具映射字典，将工具名称映射到对应的工具对象
tool_map = {tool.name: tool for tool in tools}
 
def tools_call(model_output):
    """
    根据模型输出调用相应的工具函数
    """
    # 根据模型输出中的工具名称选择对应的工具
    chosen_tool = tool_map[model_output["tool"]]
    # 使用模型输出中的参数调用选定的工具
    return chosen_tool.invoke(model_output["tool_input"])

# 调用工具函数，传入模型的输出结果
tools_call(res)

"It's 20 华氏度 in 北京"

#### Agent Planning

In [43]:
input = "123+234*345+456*567是多少"
agent_scratchpad = ""
tools_describ = []

In [44]:
# 定义乘法函数，接收两个整数参数并返回它们的乘积
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int
 
# 定义加法函数，接收两个整数参数并返回它们的和
def add(first_int: int, second_int: int) -> int:
    """add two integers together."""
    return first_int + second_int

# 将乘法函数转换为JSON格式的工具描述并添加到工具描述列表中
tools_describ.append(function_to_json(multiply))
# 将加法函数转换为JSON格式的工具描述并添加到工具描述列表中
tools_describ.append(function_to_json(add))

Multiply two integers together.
{'first_int': <class 'int'>, 'second_int': <class 'int'>, 'return': <class 'int'>}
add two integers together.
{'first_int': <class 'int'>, 'second_int': <class 'int'>, 'return': <class 'int'>}


In [45]:
tools_describ

['{\n  "name": "multiply",\n  "description": "Multiply two integers together.",\n  "parameters": {\n    "type": "object",\n    "properties": {\n      "first_int": {\n        "type": "int"\n      },\n      "second_int": {\n        "type": "int"\n      }\n    }\n  },\n  "returns": "int"\n}',
 '{\n  "name": "add",\n  "description": "add two integers together.",\n  "parameters": {\n    "type": "object",\n    "properties": {\n      "first_int": {\n        "type": "int"\n      },\n      "second_int": {\n        "type": "int"\n      }\n    }\n  },\n  "returns": "int"\n}']

In [46]:
agent_system_prompt = f"""Answer the following questions as best you can. You have access to the following tools:

{tools_describ}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{rendered_tools}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

"""

In [47]:
qwen.invoke("123+234*345+456*567是多少")

AIMessage(content='要解这个问题，我们需要遵循数学中的运算顺序规则，即先乘法后加法。所以，我们首先计算乘法部分，然后将结果相加。\n\n给定的表达式是：123 + 234 * 345 + 456 * 567\n\n首先计算乘法:\n- \\(234 * 345 = 80730\\)\n- \\(456 * 567 = 259352\\)\n\n接下来，将这些结果与123相加:\n- \\(123 + 80730 + 259352 = 340205\\)\n\n因此，123 + 234 * 345 + 456 * 567 的结果是 340205。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 188, 'prompt_tokens': 28, 'total_tokens': 216, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'id': 'chatcmpl-6ade2a92-c9c4-4a1b-9e78-1b31afd25db9', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--63de800f-004a-4925-be33-6918f8a0a210-0', usage_metadata={'input_tokens': 28, 'output_tokens': 188, 'total_tokens': 216, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

## 二、LangChain框架的主要组成部分

LangChain Libraries(库)：包括了组件的接口和集成，以及链和代理的实现，目前有python版本和JavaScript版本

LangChain Templates(模板)：官方提供的适用于各种任务的参考架构

LangServe：用于将LangChain链部署为REST API的库

LangSmith：官方提供的开发者平台，可以调试、测试、评估和监控基于任何LLM构建的链，并与LangChain无缝集成。

<img src="images/langchain_stack.svg" alt="LangChain Framework" width="100%"><br>
--<a>https://python.langchain.com/docs/get_started/introduction</a>

### LangChain1.0 重要更新
主要特点：langchain与langraph的边界进一步模糊

#### 1、create_agent API
LangChain 1.0引入了全新的create_agent抽象接口，这是对旧版多种Agent模式（如ReAct、Plan-and-Solve、React-Retry等）的统一简化

In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
import os

# 定义工具函数
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# 初始化模型 - 以DeepSeek-V3.2为例
llm = ChatOpenAI(
    base_url="https://api.siliconflow.cn/v1",
    api_key=os.getenv("SILICONFLOW_API_KEY"),
    model="deepseek-ai/DeepSeek-V3.2-Exp",
)

# 创建Agent
agent = create_agent(
    model=llm,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# 调用Agent
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]})
print(response)

#### 2、AgentMiddleware 系统
LangChain 为常见模式提供了一些预构建的中间件，包括：

（1）PIIMiddleware：在发送到模型之前编辑敏感信息，通过模式匹配自动识别并隐藏邮箱、手机号、社保号等敏感信息，助力合规开发

（2）SummarizationMiddleware：在对话历史过长时压缩对话历史记录

（3）HumanInTheLoopMiddleware：敏感工具调用需要批准，即在执行工具前暂停，等待用户确认或修改，特别适合涉及外部系统调用或敏感操作的场景（如自动发邮件、转账）

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import (
PIIMiddleware,
SummarizationMiddleware,
HumanInTheLoopMiddleware
)

agent = create_agent(
    model="claude-sonnet-4-5-20250929",
    tools=[read_email, send_email],
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware(
            "phone_number",
            detector=(
                r"(?:\+?\d{1,3}[\s.-]?)?"
                r"(?:\(?\d{2,4}\)?[\s.-]?)?"
                r"\d{3,4}[\s.-]?\d{4}"
            ),
            strategy="block"
        ),
        SummarizationMiddleware(
            model="claude-sonnet-4-5-20250929",
            max_tokens_before_summary=500
        ),
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email": {
                    "allowed_decisions": ["approve", "edit", "reject"]
                }
            }
        ),
    ]
)

我们还可以构建自定义中间件以满足特定需求。中间件在代理执行的每个步骤中公开钩子：

<img src="images/AgentMiddleware.png" alt="AgentMiddleware" width="80%"><br>

通过在 AgentMiddleware 类的子类上实现以下任何Hook来构建自定义中间件：
|Hook|运行时|使用案例|
|--|--|--|
|before_agent|Before calling the agent|加载内存，验证输入|
|before_model|Before each LLM call|更新提示，修剪消息|
|wrap_model_call|Around each LLM call|拦截和修改请求/响应|
|wrap_tool_call|Around each tool call|拦截和修改工具执行|
|after_model|After each LLM response|验证输出，应用护栏|
|after_agent|After agent completes|保存结果、清理|


自定义中间件示例：

In [1]:
from dataclasses import dataclass
from typing import Callable

from langchain_openai import ChatOpenAI

from langchain.agents.middleware import (
AgentMiddleware,
ModelRequest
)
from langchain.agents.middleware.types import ModelResponse

@dataclass
class Context:
    user_expertise: str = "beginner"

class ExpertiseBasedToolMiddleware(AgentMiddleware):
    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse]
    ) -> ModelResponse:
        user_level = request.runtime.context.user_expertise

        if user_level == "expert":
            # More powerful model
            model = ChatOpenAI(model="gpt-5")
            tools = [advanced_search, data_analysis]
        else:
            # Less powerful model
            model = ChatOpenAI(model="gpt-5-nano")
            tools = [simple_search, basic_calculator]

        request.model = model
        request.tools = tools
        return handler(request)

agent = create_agent(
    model="claude-sonnet-4-5-20250929",
    tools=[
        simple_search,
        advanced_search,
        basic_calculator,
        data_analysis
    ],
    middleware=[ExpertiseBasedToolMiddleware()],
    context_schema=Context
)

ModuleNotFoundError: No module named 'langchain.agents.middleware'

有关详细信息，请参阅完整的中间件指南：https://docs.langchain.com/oss/python/langchain/middleware

#### 3、Content Blocks
LangChain 1.0在langchain-core中新增了content_blocks属性，为所有模型输出定义了统一标准。这一改进解决了不同模型提供商输出格式差异导致的兼容性问题，使得切换模型后，流式输出、UI渲染、记忆存储等下游逻辑无需修改即可继续工作

```python
ContentBlock = (
    TextContentBlock #纯文本输出
    | InvalidToolCall #处理工具调用失败的情况
    | ReasoningContentBlock #模型的推理思考过程展示
    | NonStandardContentBlock #灵活的备用选项，用于容纳不符合任何标准分类的特殊内容
    | DataContentBlock #处理文本之外的结构化数据或非文本内容（如图像）
    | ToolContentBlock #模型对外部工具的调用请求
)
```

#### 4、Structured Output
Structured Output（结构化输出）是一个极其重要的功能，它能确保大语言模型（LLM）的输出是结构化的数据，例如JSON对象或Pydantic模型实例，而不是非结构化的文本，这大大简化了下游程序对模型结果的处理和集成

with_structured_output()是LangChain官方推荐的、最简单可靠的方法，它的底层原理是优先使用LLM原生的函数调用能力，通过定义好Pydantic模型来精确描述你希望得到的数据结构，让LLM具备结构化输出的能力

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# 定义期望的输出结构
class PersonInfo(BaseModel):
    name: str = Field(description="人物的全名")
    age: int = Field(description="人物的年龄", ge=0, le=150)  # 年龄范围约束
    occupation: str = Field(description="职业")
    hobbies: list[str] = Field(description="爱好列表")

# 使用结构化输出
llm = ChatOpenAI(model="gpt-3.5-turbo")
structured_llm = llm.with_structured_output(PersonInfo)

result = structured_llm.invoke("描述一个30岁的软件工程师，喜欢徒步和摄影")
print(result)
# 输出: PersonInfo(name="张三", age=30, occupation="软件工程师", hobbies=["徒步", "摄影"])

它也可以与链式进行组合

with_structured_output 方法支持两种模式，可以通过 method 参数指定：

（1）method="function_calling" (默认)：利用模型的函数调用功能，这是最稳定和推荐的方式。

（2）method="json_mode"：使用模型的JSON输出模式。但并非所有模型都支持此模式，且可能需要在Prompt中明确指示输出JSON结构

In [ ]:
from langchain_openai import ChatOpenAI

# 初始化模型
llm = ChatOpenAI(model="gpt-3.5-turbo")

# 为核心模型绑定结构化输出能力
structured_llm = llm.with_structured_output(Joke)  # 默认使用 'function_calling' 模式

# 直接调用
result = structured_llm.invoke("讲一个关于猫的笑话")
print(result)
# 输出: Joke(setup="为什么猫会打开冰箱？", punchline="因为它想喝点'喵'料！")

# 也可以轻松融入链中
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个专业的喜剧演员。"),
    ("human", "{input}")
])
chain = {"input": RunnablePassthrough()} | prompt | structured_llm
print(chain.invoke("讲一个关于编程的笑话"))
# 使用JSON模式 (如果模型支持，如 gpt-4o)
structured_llm_json = llm.with_structured_output(Joke, method="json_mode")

### LangChain组件拆解

LangChain的工作流程如下图所示：

<img src="images/langchain workflow.jpg" alt="LangChain workflow" width="80%"><br>
 
接下来我们将着重介绍框架图中集成组件（integrations components）和协议（protocol）的主要内容。

#### 2.1、LangChain Expression Language (LCEL)

LangChain Expression Language （LCEL, LangChain表达式语言）是一种声明式的方法，可以容易的将组件链接起来。LCEL语言的目的是为了简化组件的链接，使得开发者可以更加专注于业务逻辑的实现，而不是组件的链接。通过LCEL，开发者可以将组件链接起来，形成一个链，然后将链部署为一个服务，供外部调用。

LCEL的核心能力包括：
- 流媒体支持(Streaming support)：可以处理流媒体数据，而不仅仅是文本。
- 异步支持(Async support)：可以异步执行，而不仅仅是同步执行。
- 优化的并行处理(Optimized parallel execution)：自动并行处理，以提高性能。
- 重试和回退机制(Retries and fallbacks)：可以在链的任何位置进行重试和回退。
- 访问中间结果(Access intermediate results)：可以访问链中间的任何结果。
- 输入和输出架构(Input and output schemas)：可以提供Pydantic模式和JSONSchema模式的输入和输出，便于验证。
- 无缝LangSmith集成(Seamless LangSmith tracing integration)：使用LCEL，步骤会自动记录到LangSmith中，便于调试和监控。
- 无缝LangServe集成(Seamless LangServe deployment integration)：使用LCEL创建的链都可以使用LangServe部署。

<div style="background-color: #ffffcc; border: 1px solid #93c47d; padding: 10px; border-radius: 5px; margin-bottom: 20px;">
    <ul>文中使用千问替代openai。langchain中使用千问或其他国内模型时，通常有两种方式，一种是直接使用对应的库，如千问对应的是from langchain_community.chat_models.tongyi import ChatTongyi或者以兼容Openai的形式来调用，但应注意的是，不同模型的embedding通常不兼容，在开发带有memory功能的项目时，要注意区分。<br>千问等国内比较有影响力的大模型通常被langchain兼容，但兼容程度不一定很完美，而semantic kernel则未针对国内大模型进行开发，因此只能使用openai的调用方式来调用国内大模型，兼容性较差。<br>在本教材中以openai的兼容用法为主。</ul>
</div>

**LCEL演示**

下面用一个示例，让大家初步感受一下LCEL

In [114]:
# !pip install -r requirements.txt
# !pip list
#若下载安装较慢，可参照使用以下语句
#!pip install -r requirements.txt -i https://pypi.tuna.tsinghua.edu.cn/simple

In [115]:
# !pip uninstall -y openai
# !pip install openai

In [116]:
# !pip list

In [49]:
# 导入dotenv模块
import dotenv
import os

# 加载.env文件中的环境变量
dotenv.load_dotenv("../.env")

True

定义链

In [50]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# 创建一个聊天提示模板，要求生成关于特定主题的短笑话
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# 使用qwen-max模型进行聊天
model = ChatOpenAI(
    model="qwen-max",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1"
)
# model = ChatTongyi(model="qwen-max")
# 创建一个字符串输出解析器
output_parser = StrOutputParser()

# 使用LCEL链式调用，这是LCEL定义链的方式，可以用简单拼接的方式实现chain的定义
chain = prompt | model | output_parser

In [51]:
# model.invoke("你好")

当定义了一个chain后，下面我们对比一下使用LCEL和不用LCEL的不用操作。当前大家可以先整体感受一下区别，等学习完LangChain的核心组件后，再返回来细看。

Invoke （调用chain操作）

In [55]:
#使用LCEL链式调用
from langchain_core.runnables import RunnablePassthrough

# 创建一个聊天提示模板，要求生成关于特定主题的短中文笑话
prompt = ChatPromptTemplate.from_template(
    "Tell me a short chinese joke about {topic} in Chinese."
)

# 创建一个字符串输出解析器
output_parser = StrOutputParser()

# 使用qwen-max模型进行聊天
model = ChatOpenAI(
    model="qwen-max",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1"
)
# 创建一个LCEL链式调用，将参数传递给RunnablePassthrough对象
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

# 调用链式调用，生成关于"ice cream"主题的中文笑话
chain.invoke("ice cream")

'当然可以，这里有一个关于冰淇淋的简短中文笑话：\n\n为什么冰淇淋不会生气？\n\n因为它很“冷”静！（这里的“冷静”是双关语，既有冷静的意思，也有冷的意思。）'

In [53]:
#不使用LCEL链式调用
from typing import List
import openai

# 定义聊天提示模板
prompt_template = "Tell me a short chinese joke about {topic} in Chinese."

# 创建 OpenAI 客户端
client = openai.OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),#调整为qwen的key
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"#调整为qwen的url
)
    # client = OpenAI(
    #     api_key=os.getenv("DASHSCOPE_API_KEY"),
    #     base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    # )

# 调用聊天模型函数，传入消息并返回生成的文本
def call_chat_model(messages: List[dict]) -> str:
    response = client.chat.completions.create(
        model="qwen-max",#改成qwen的模型名 
        messages=messages,
    )
    return response.choices[0].message.content

# 执行链式调用函数，生成关于特定主题的中文笑话
def invoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return call_chat_model(messages)

# 调用函数，生成关于"ice cream"主题的中文笑话
invoke_chain("ice cream")

'当然可以，这里有一个关于冰淇淋的简短中文笑话：\n\n为什么冰淇淋不会生气？\n\n因为它很冷（冷静）！\n\n这个笑话利用了“冷”这个词的一语双关，既指温度低也指情绪上的冷静。希望你喜欢！'

Stream （流式操作）

In [54]:
#使用LCEL链式调用
# 遍历链式调用的流式输出，生成关于"ice cream"主题的中文笑话并逐块打印输出
for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

当然可以，这里有一个关于冰淇淋的简短中文笑话：

为什么冰淇淋不会生气？

因为它很冷（冷静）！

这个笑话利用了“冷”字的双关意思，在中文里，“冷”既指温度低，也有冷静、不生气的意思。希望你喜欢！

In [123]:
#不使用LCEL链式调用
from typing import Iterator

# 定义流式聊天模型函数，接受消息列表并返回生成文本的迭代器
def stream_chat_model(messages: List[dict]) -> Iterator[str]:
    # 创建流式聊天模型
    stream = client.chat.completions.create(
        model="qwen-max",
        messages=messages,
        stream=True,
    )
    # 遍历流式输出
    for response in stream:
        content = response.choices[0].delta.content
        if content is not None:
            yield content

# 定义流式链式调用函数，生成关于特定主题的中文笑话的迭代器
def stream_chain(topic: str) -> Iterator[str]:
    prompt_value = prompt_template.format(topic=topic)
    return stream_chat_model([{"role": "user", "content": prompt_value}])

# 遍历流式生成的关于"ice cream"主题的中文笑话并逐块打印输出
for chunk in stream_chain("ice cream"):
    print(chunk, end="", flush=True)


当然可以，这里有一个关于冰淇淋的简短中文笑话：

为什么冰淇淋不会生气？

因为它很“冷”静！（“冷静”在中文里是保持镇定的意思，“冷”字也指温度低，这里巧妙地使用了双关。）

Batch （批量/并行处理）

In [124]:
#使用LCEL链式调用
# 使用链式调用批量处理多个主题，生成关于每个主题的中文笑话
chain.batch(["ice cream", "spaghetti", "dumplings"])

['当然可以，这里有一个关于冰淇淋的简短中文笑话：\n\n为什么冰淇淋不会生气？\n\n因为它很冷（冷静）！\n\n这个笑话利用了“冷”这个词的双关意思，既指冰淇淋的物理特性，也用来形容人冷静、不生气的状态。希望你喜欢！',
 '当然可以，这里有一个关于意大利面的简短中文笑话：\n\n为什么意大利面要去上学？\n\n因为它想变得更有“面”儿（有面子）！\n\n这个笑话利用了“面”字的双关意思，在中文里，“面”既可以指面条，也可以指面子、尊严。希望你觉得有趣！',
 '当然可以，这里有一个关于饺子的简短笑话：\n\n为什么饺子很有学问？\n因为它有“内涵”！\n\n解释一下，“内涵”在中文里既可以指饺子里面的馅料，也可以指一个人有深度、有内容。这个笑话就是利用了这个词的双关意思。希望你喜欢！']

In [125]:
#不使用LCEL链式调用
from concurrent.futures import ThreadPoolExecutor

def batch_chain(topics: list) -> list:
    # 使用ThreadPoolExecutor创建一个最大工作线程数为5的线程池
    with ThreadPoolExecutor(max_workers=5) as executor:
        # 调用executor.map方法，对topics中的每个topic调用invoke_chain方法，返回结果作为列表
        return list(executor.map(invoke_chain, topics))

batch_chain(["ice cream", "spaghetti", "dumplings"])

['当然可以，这里有一个关于冰淇淋的简短笑话：\n\n为什么冰淇淋不会生气？\n\n因为它很冷（冷静）！\n\n用中文来说就是：\n为什么冰淇淋不会生气？\n因为它很冷静！（“冷静”在这里既有“冷静”的意思，也暗示了冰淇淋是冷的。）',
 '当然可以，这里有一个关于意大利面的简短中文笑话：\n\n为什么意大利面要去上学？\n\n因为它想变得更有“面条”（头脑）！\n\n解释一下，“面条”在中文里发音与“头脑”相似，所以这个笑话是通过谐音来制造幽默感的。希望你喜欢！',
 '当然可以，这里有一个关于饺子的简短笑话：\n\n为什么饺子去了游泳池却没游呢？\n因为它怕被“煮”了！\n\n这个笑话利用了“煮”和“游”的谐音来制造幽默感。希望你喜欢！']

Async （异步）

In [126]:
#使用LCEL链式调用
#异步调用
chain.ainvoke("ice cream")

<coroutine object RunnableSequence.ainvoke at 0x7f017bb8a5e0>

In [127]:
#不使用LCEL链式调用

# 创建一个异步OpenAI客户端
async_client = openai.AsyncOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),#调整为qwen的key
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"#调整为qwen的url

)

# 异步函数，用于调用OpenAI的聊天模型
async def acall_chat_model(messages: List[dict]) -> str:
    # 调用OpenAI的聊天模型完成对话
    response = await async_client.chat.completions.create(
        model="qwen-max", 
        messages=messages,
    )
    # 返回模型生成的响应消息内容
    return response.choices[0].message.content

# 异步函数，用于处理传入的主题
async def ainvoke_chain(topic: str) -> str:
    # 根据传入的主题生成对话模型的输入模板
    prompt_value = prompt_template.format(topic=topic)
    # 构造对话所需的消息列表
    messages = [{"role": "user", "content": prompt_value}]
    # 调用聊天模型并返回生成的响应
    return await acall_chat_model(messages)

In [128]:
#异步运行
await ainvoke_chain("ice cream")

'当然可以，这里有一个关于冰淇淋的简短中文笑话：\n\n为什么冰淇淋不会生气？\n\n因为它很冷（冷静）！\n\n这个笑话利用了“冷”字的双关意思，在中文里，“冷”既可以指温度低，也可以用来形容人很冷静、不生气。希望你喜欢这个笑话！'

Logging （开启LangSmith，并查看logging）

In [129]:
import os
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['LANGCHAIN_PROJECT'] = "langchain"
# 已经设置了系统环境变量，不需要再设置
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"


In [130]:
chain.invoke("ice cream")

'当然可以，这里有一个关于冰淇淋的简短中文笑话：\n\n为什么冰淇淋不会生气？\n\n因为它很冷（冷静）！\n\n这个笑话利用了“冷”这个词的一语双关，既指温度上的冷，也指情绪上的冷静。希望你喜欢！'

以上展示了LCEL的基本使用方式，可以看出LCEL提供了一些十分便利的操作方式，并且可以更加灵活的设计应用的逻辑。

**LCEL实现原理**
LCEL语言中使用的“|”称为管道运算符，通过这种方式，可以“直观地”构建整个逻辑链条，更快速书写且更易读。但可能很多人第一次见到这种用法，很难理解，其实这是面向对象编码中的运算符重载，可以对运算符，进行自定义，使其实现额外的功能。下面我们可以模拟langchain的调用方式，自己构建一个管道运算符。

In [59]:
class MyPipeline:
    #类的初始化
    def __init__(self, func):
        self.func = func
    #运算符重载
    def __or__(self, otherfunc):
        def _func(*args, **kwargs):
            return otherfunc(self.func(*args, **kwargs))
        return MyPipeline(_func)
        
    #常规调用方法
    def __call__(self, *args, **kwargs) :
        return self.func(*args, **kwargs)
        
    #invoke调用方法
    def invoke(self, *args, **kwargs):
        return self.__call__(*args, **kwargs)

In [60]:
def my_function(*args):
    for arg in args:
        print(arg)

my_function(1, 2, 3)

1
2
3


In [61]:
def my_function(**kwargs):
    for key, value in kwargs.items():
        print(f"{key}: {value}")

my_function(a=1, b=2, c=3)

a: 1
b: 2
c: 3


In [62]:
from datetime import datetime

def hello(name) -> str:
    return f"hello，我是{name}。"
 
def welcome(greeting) -> str:
    return f"欢迎使用管道运算符, {greeting}"
 
def attach_date(mstr) -> str:
    return f'{mstr} 现在是{datetime.now().strftime("%Y-%m")}。'
 
hello = MyPipeline(hello)
welcome = MyPipeline(welcome)
attach_date = MyPipeline(attach_date)

In [63]:
my_chain = hello.__or__(welcome).__or__(attach_date)
print(my_chain.invoke("zoey"))

欢迎使用管道运算符, hello，我是zoey。 现在是2025-09。


In [136]:
my_chain = hello | welcome | attach_date
print(my_chain.invoke("zoey"))

欢迎使用管道运算符, hello，我是zoey。 现在是2025-09。


#### 2.2、核心组件
LangChain提供了标准的、可扩展的接口和外部集成，最主要包括：Model I/O、Retrieval、Agents等。下面将对这些组件进行重点介绍。以下程序主要来自于官方文档，采用0.0.350版本的langchain库。

**Model I/O**

Model I/O是LangChain的核心组件，它提供了一种标准的接口，用于将语言模型与其他组件集成。主要包括4个部分：Prompts、Chat models、LLMs和Output parsers。其工作流程如下图所示：

<img src="images/model_IO.jpg" alt="Model I/O" width="60%"><br>
--<a>https://python.langchain.com/docs/modules/model_io/</a>


- Prompts 提示：语言模型的提示是用户提供的信息，用于指导模型生成响应。提示可以是文本、聊天消息、图像、视频、音频或其他任何内容。Prompts 提供了一种标准的接口，用于将提示与语言模型集成。Prompts 有助于将用户输入和其他动态信息转换为适合语言模型的格式。LangChain提供了许多类和函数来辅助创建和使用Prompts。
 
    包括Prompt templates和Example selectors两大类:
 
    - Prompt templates是LangChain提供的创建和使用提示模板的工具，除一些常用的默认模板外，还支持将提示连接到特征库、自定义模板、设置输出格式、从文件加载提示等功能；
 
    - 而当输入的提示信息量较大时，可以使用Example selectors来动态的选择提示信息，以便于模型更好的理解用户的输入，包括自定义选择器、基于长度的选择器、基于MMR的选择器、基于相似度的选择器等。
  

<div class="alert alert-success">
①使用PromptTemplate来创建字符串提示模板：
默认使用str.format的方式进行格式化
</div>


In [64]:
# from langchain.prompts import PromptTemplate#用这个不能建链
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 使用PromptTemplate类的from_template方法创建一个模板
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content} in Chinese."
)

adjective = "funny"
content = "chickens"

examp = f"Tell me a {adjective} joke about {content} in Chinese."
print(examp)
# 格式化模板，替换adjective为"funny"，content为"chickens"
prompt = prompt_template.format(adjective="funny", content="chickens")
print(prompt)

Tell me a funny joke about chickens in Chinese.
Tell me a funny joke about chickens in Chinese.


In [65]:
output_parser = StrOutputParser()

# 创建ChatOpenAI实例，指定模型为"qwen-max"
model = ChatOpenAI(
    model="qwen-max",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

# 创建处理链，包括RunnablePassthrough、prompt_template、model和output_parser
chain = (
    {"adjective": RunnablePassthrough(), "content": RunnablePassthrough()} 
    | prompt_template
    | model
    | output_parser
)

# 调用处理链，传入包含两个元素的列表作为输入
chain.invoke(["sad", "ice cream"])

'当然可以。这里有一个关于“悲伤”和“冰淇淋”的笑话：\n\n为什么冰淇淋会感到悲伤？\n\n因为它总是被冷落，而且还没人能长久地留住它的心——每次人们都只是一下子就把它吃掉了。\n\n这个笑话用中文来说就是：\n为什么冰淇淋会感到难过？\n因为它总是被人冷落，而且还没人能够长久地留住它的心——每次人们都只是一下子就把它吃光了。'

In [66]:
# 从模板创建PromptTemplate实例，模板包含占位符{adjective}和{content}
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content} in Chinese."
)

# 格式化PromptTemplate实例，替换占位符{adjective}为"funny"，{content}为"chickens"
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens in Chinese.'

In [67]:
# 创建处理链，包括prompt_template、model和output_parser
chain = prompt_template | model | output_parser

# 调用处理链，传入一个包含"adjective"和"content"键的字典作为输入
chain.invoke({"adjective": "funny", "content": "chickens"})



'当然可以，这里有一个关于鸡的笑话：\n\n为什么鸡要过马路？\n\n因为它想到另一边去“咯”！\n\n这个笑话利用了中文里“咯”这个词，它既可以表示鸡叫声，也有“说”的意思。希望这个笑话能让你会心一笑！'

也可以没有变量，直接使用提示模板：

In [141]:
from langchain_core.prompts import PromptTemplate

# 从模板创建PromptTemplate实例，模板内容为"Tell me a joke"
prompt_template = PromptTemplate.from_template("Tell me a joke")

# 格式化PromptTemplate实例，返回格式化后的字符串
prompt_template.format()

'Tell me a joke'

In [142]:
# 创建处理链，包括prompt_template、model和output_parser
chain = prompt_template | model | output_parser

# 调用处理链，传入一个空字典作为输入
chain.invoke({})

"Sure, here's a light joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"

<div class="alert alert-success">
②使用ChatPromptTemplate创建聊天消息提示模板：
</div>

In [68]:
from langchain_core.prompts import ChatPromptTemplate

# 从消息列表创建ChatPromptTemplate实例
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing {name}?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

# 格式化消息模板，传入name和user_input参数，并将结果存储在messages变量中
messages = chat_template.format_messages(name="Bob",user_input="What is your name?")

messages

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you doing Bob?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]

In [69]:
# 创建处理链，包括chat_template、model和output_parser
chain = chat_template | model | output_parser

# 调用处理链，传入包含name和user_input键值对的字典作为输入
chain.invoke({"name":"Zoey","user_input":"你的名字是什么"})

'我的名字是Zoey。你好！有什么可以帮助你的吗？'

<div class="alert alert-success">
③使用FewShotPromptTemplate创建聊天用于few-shot的提示模板：
首先定义一个few-shot的示例列表：
</div>

In [71]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
# 定义包含问题和答案的示例列表
examples = [
  {
    "question": "Who lived longer, Muhammad Ali or Alan Turing?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
"""
  },
  {
    "question": "When was the founder of craigslist born?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
"""
  },
  {
    "question": "Who was the maternal grandfather of George Washington?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
  },
  {
    "question": "Are both the directors of Jaws and Casino Royale from the same country?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
  }
]

然后创建一个few-shot的格式化器（formatter）：

In [72]:
# 创建PromptTemplate实例，指定输入变量为"question"和"answer"，定义模板格式为"Question: {question}\n{answer}"
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

# 格式化并打印第一个示例的问题和答案内容
print(example_prompt.format(**examples[0]))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



最后将示例和格式化器传入FewShotPromptTemplate中：

In [73]:
# 创建FewShotPromptTemplate实例，传入示例列表、示例模板、后缀和输入变量
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)
# 格式化并打印输入问题"Who was the father of Mary Ball Washington?"的答案
print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

In [74]:
# 创建处理链，依次包含prompt、model和output_parser
chain = prompt | model | output_parser
# 调用处理链，输入问题"万达老板的儿子叫什么"并获取输出
chain.invoke({"input":"万达老板的儿子几岁了"})

'要回答这个问题，我们需要明确几个信息点：万达老板是谁，他的儿子叫什么名字，以及他儿子的出生年份或当前年龄。让我们分步骤来解答。\n\nFollow up: 万达的老板是谁？\nIntermediate answer: 万达集团的创始人和主要拥有者是王健林。\n\nFollow up: 王健林的儿子叫什么名字？\nIntermediate answer: 王健林的儿子名叫王思聪。\n\nFollow up: 王思聪出生于哪一年？\nIntermediate answer: 王思聪出生于1988年1月3日。\n\n根据这些信息，我们可以计算出王思聪的年龄（基于当前年份）。假设现在是2023年，那么王思聪大约35岁了。\n- 如果查询时的具体年份不同，请相应调整计算方法以得出准确年龄。\n\n所以，最终答案取决于具体的查询时间，但基于上述信息，在2023年时，王健林的儿子王思聪大约35岁。'

当我们有大量示例时，可能需要对这些示例进行挑选，以便组装出效果较好的Prompt，此时可以借助LangChain构建一个ExampleSelector。目前LangChain集成了4种选择器，分别是Similarity、MMR、Length和Ngram四种方式，用来挑出某种程度上最接近的示例。此外，LangChain也支持自定义选择器。

<div class="alert alert-success">
④创建一个基于长度的ExampleSelector：
</div>
下面是一个基于长度的选择器，当参数adjective的长度小于25时，将命中所有选项，否则将只命中第一个选项

In [75]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector


# 虚构任务示例：创建反义词
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

# 创建示例模板，指定输入变量为"input"和"output"，定义模板格式为"Input: {input}\nOutput: {output}"
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# 创建基于长度的示例选择器，传入示例、示例模板、最大长度
example_selector = LengthBasedExampleSelector(
    # The examples it has available to choose from.
    examples=examples,
    # The PromptTemplate being used to format the examples.
    example_prompt=example_prompt,
    # The maximum length that the formatted examples should be.
    # Length is measured by the get_text_length function below.
    max_length=15,
    # The function used to get the length of a string, which is used
    # to determine which examples to include. It is commented out because
    # it is provided as a default value if none is specified.
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)

# 创建FewShotPromptTemplate实例，传入示例选择器、示例模板、前缀、后缀和输入变量
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [76]:
# 一个小输入示例，因此它会选择所有示例。
print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: big
Output:


In [77]:
# 定义一个长字符串
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
# 使用动态字符串格式化并打印
print(dynamic_prompt.format(adjective=long_string))

Give the antonym of every input

Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Output:


同样可以增加选项

In [78]:
# 创建一个新的示例，将"big"映射为"small"
new_example = {"input": "big", "output": "small"}
# 将新示例添加到示例选择器中
dynamic_prompt.example_selector.add_example(new_example)
# 使用动态提示格式化字符串，指定形容词为"enthusiastic"
print(dynamic_prompt.format(adjective="enthusiastic"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: enthusiastic
Output:


<div class="alert alert-success">
⑤创建一个自定义的ExampleSelector：
</div>

In [79]:
from langchain.prompts.example_selector.base import BaseExampleSelector
from typing import Dict, List
import numpy as np

# 创建一个自定义的示例选择器类
class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples: List[Dict[str, str]]):
        self.examples = examples
    
    def add_example(self, example: Dict[str, str]) -> None:
        """添加新示例
        到存储中对应的键值。"""
        self.examples.append(example)

    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """根据输入选择要使用的示例。"""
        return np.random.choice(self.examples, size=2, replace=False)


初始化一个简单的示例

In [80]:
# 创建示例列表
examples = [
    {"foo": "1"},
    {"foo": "2"},
    {"foo": "3"}
]

# 初始化示例选择器
example_selector = CustomExampleSelector(examples)

可以看到选择过程中只能选到1、2、3的例子

In [86]:
# 选择示例（只能选择1、2、3）
example_selector.select_examples({"foo": "foo"})

array([{'foo': '3'}, {'foo': '1'}], dtype=object)

增加一个选项

In [87]:
# 将新示例添加到示例集合中
example_selector.add_example({"foo": "4"})
example_selector.examples

[{'foo': '1'}, {'foo': '2'}, {'foo': '3'}, {'foo': '4'}]

现在可以选到第4个选项了

In [94]:
# 选择示例（现在可以选到4了）
example_selector.select_examples({"foo": "foo"})

array([{'foo': '4'}, {'foo': '2'}], dtype=object)

**Agents**
<div class="alert alert-success">
AI领域中的agent通常指一种旨在与现实世界交互的实体，该实体包括AI核心、感知器、作用器等部件，通常也翻译为智能体，如“阿法狗”（alphaGO）就是一个agent。
</div>
LangChain中的Agents类，提供了一套完整的工具和方法，可快速构建agent。这里我们讨论的Agents其AI核心为大语言模型。

Agents智能体的核心思想是使用语言模型来推动决策执行。Agent可以访问一组工具，并可以根据用户输入决定调用哪个工具。相关的主要概念包括：
- Agent：应用程序的逻辑核心，主要由LLM和prompt赋能，它接受用户输入以及代理以前执行的步骤列表，并返回AgentAction（代理动作）或AgentFinish（代理完成）。
- Tools：是指Agents可以执行的操作，这取决于APP想要实现什么功能
- Toolkits：对某些问题，LangChain提供了一些相关工具
- AgentExecutor：是agent的运行时（runtime），它将代理和工具列表包装在一起。它负责迭代运行代理，直到满足停止条件。

接下来我们使用官网样例从零开始创建一个Agent，以便更好的理解Agent的工作原理。
还有一些会有到的概念：
- AgentAction：代理动作，是指代理执行的操作，如调用工具、执行其他代理等
- AgentFinish：代理完成，是指代理执行完毕，不再执行其他操作
- intermediate_steps：中间步骤，是指代理执行的中间步骤，如调用工具、执行其他代理等

<div class="alert alert-success">
①定义一个Agent：
</div>

先定义聊天模型，通过调用模型，我们可以看到，chatgpt对于字符串长度，并不能得到正确答案

In [96]:
from langchain.chat_models import ChatOpenAI

# 创建一个ChatOpenAI对象，指定模型为"qwen"，温度为0
llm = ChatOpenAI(
    model="qwen-max",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1"
)
# 使用invoke方法向模型提问
llm.invoke("how many letters in the word qwejfnvjdkvmcnsdjasdf?").content

'The word "qwejfnvjdkvmcnsdjasdf" contains 24 letters.'

In [168]:
from langchain.chat_models import ChatOpenAI

# 创建一个ChatOpenAI对象，指定模型为"qwen"，温度为0
llm = ChatOpenAI(
    model="deepseek-chat", 
    api_key="sk-fdb9c87b872e4dce8efe0b5667435777",
    openai_api_base="https://api.deepseek.com/v1",    
    temperature=0)
# 使用invoke方法向模型提问
llm.invoke("how many letters in the word qwejfnvjdkvmcnsdjasdf?").content
#-> 但是答案并不总是准确的。

#define a simple Python function as a tool.
from langchain.agents import tool

# 使用tool装饰器定义一个简单的Python函数作为工具
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)
    
# 将get_word_length函数添加到工具列表中
tools = [get_word_length]

#装饰器
def simple_decorator(func):
    def wrapper(*args, **kwargs):
        print("函数前")
        result = func(*args, **kwargs)
        print("函数后")
        return result
    return wrapper

# 使用simple_decorator装饰器来包装say_hello函数
@simple_decorator
def say_hello(name):
    print(f"你好, {name}!")
@simple_decorator
def say_bye():
    print("拜拜👋")
# 使用装饰器后，say_hello 函数的实际行为被简单地包裹了
say_bye()

#Now let us create the prompt. 
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# 从消息创建聊天提示模板
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        ("user", "{input}"),
        # ("user","how many letters in the word educa?"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

from langchain.tools.render import format_tool_to_openai_function
# 将工具格式化为OpenAI函数并绑定到LLM
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
# 创建代理
agent = (
    {
         # 定义输入的处理方式
        "input": lambda x: x["input"],       
        # 定义 agent_scratchpad 的处理方式，将中间步骤格式化为 OpenAI 函数消息
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
)

/tmp/ipykernel_23277/2173003741.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


函数前
拜拜👋
函数后


/tmp/ipykernel_23277/2173003741.py:62: LangChainDeprecationWarning: The function `_format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])


下面定义计算字符串长度的函数，并将其设置为tool

In [105]:
#define a simple Python function as a tool.
from langchain.agents import tool

# 使用tool装饰器定义一个简单的Python函数作为工具
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""  # 将两个整数相乘
    return first_int * second_int
 
# 使用 @tool 装饰器定义加法工具函数
@tool
def add(first_int: int, second_int: int) -> int:
    """add two integers together."""  # 将两个整数相加
    return first_int + second_int

# 将get_word_length函数添加到工具列表中
tools = [get_word_length,multiply,add]


In [98]:
#装饰器
def simple_decorator(func):
    def wrapper(*args, **kwargs):
        print("函数前")
        result = func(*args, **kwargs)
        print("函数后")
        return result
    return wrapper

# 使用simple_decorator装饰器来包装say_hello函数
@simple_decorator
def say_hello(name):
    print(f"你好, {name}!")
@simple_decorator
def say_bye():
    print("拜拜👋🏻")
# 使用装饰器后，say_hello 函数的实际行为被简单地包裹了
say_bye()

函数前
拜拜👋🏻
函数后


定义Prompt模板

In [106]:
#Now let us create the prompt. 
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# 从消息创建聊天提示模板
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        ("user", "{input}"),
        # ("user","how many letters in the word educa?"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

将定义的工具与模型进行绑定

In [107]:
from langchain.tools.render import format_tool_to_openai_function
# 将工具格式化为OpenAI函数并绑定到LLM
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

In [108]:
# 输出LLM_with_tools的kwargs属性
llm_with_tools.kwargs

{'functions': [{'name': 'get_word_length',
   'description': 'Returns the length of a word.',
   'parameters': {'properties': {'word': {'type': 'string'}},
    'required': ['word'],
    'type': 'object'}},
  {'name': 'multiply',
   'description': 'Multiply two integers together.',
   'parameters': {'properties': {'first_int': {'type': 'integer'},
     'second_int': {'type': 'integer'}},
    'required': ['first_int', 'second_int'],
    'type': 'object'}},
  {'name': 'add',
   'description': 'add two integers together.',
   'parameters': {'properties': {'first_int': {'type': 'integer'},
     'second_int': {'type': 'integer'}},
    'required': ['first_int', 'second_int'],
    'type': 'object'}}]}

定义（或设置）agent

In [109]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
# 创建代理
agent = (
    {
         # 定义输入的处理方式
        "input": lambda x: x["input"],       
        # 定义 agent_scratchpad 的处理方式，将中间步骤格式化为 OpenAI 函数消息
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)


调用agent

In [103]:
# 调用代理并传入输入信息和中间步骤
agent.invoke({"input": "how many letters in the word qwksjfedsazxc?", "intermediate_steps": []})

AgentActionMessageLog(tool='get_word_length', tool_input={'word': 'qwksjfedsazxc'}, log="\nInvoking: `get_word_length` with `{'word': 'qwksjfedsazxc'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"word": "qwksjfedsazxc"}', 'name': 'get_word_length'}}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 189, 'total_tokens': 212, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--a84a5ae4-18ba-4df9-a1c5-404b8841baae-0')])

<div class="alert alert-success">
②创建Agent的runtime
</div>


创建一个最简单的runtime，循环调用agent，然后执行操作，直到agent返回AgentFinish为止：


In [113]:
from langchain.schema.agent import AgentFinish

# 用户输入和中间步骤初始化
user_input = "123*234+345*456+567*678是多少"
intermediate_steps = []
# 循环调用代理
while True:
    # 调用代理并传入输入信息和中间步骤
    output = agent.invoke(
        {
            "input": user_input,
            "intermediate_steps": intermediate_steps,
        }
    )
    # 判断输出类型是否为 AgentFinish
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        # 打印工具名称和工具输入
        print(f"TOOL NAME: {output.tool}")
        print(f"TOOL INPUT: {output.tool_input}")
        # 根据工具名称选择对应的函数
        tool = {"get_word_length": get_word_length, 'multiply':multiply, 'add':add}[output.tool]
        # 运行工具函数并记录中间步骤
        observation = tool.run(output.tool_input)
        print(observation)
        intermediate_steps.append((output, observation))
# 打印最终结果
print(final_result)

TOOL NAME: multiply
TOOL INPUT: {'first_int': 123, 'second_int': 234}
28782
TOOL NAME: multiply
TOOL INPUT: {'first_int': 345, 'second_int': 456}
157320
TOOL NAME: multiply
TOOL INPUT: {'first_int': 567, 'second_int': 678}
384426
TOOL NAME: add
TOOL INPUT: {'first_int': 28782, 'second_int': 157320}
186102
TOOL NAME: add
TOOL INPUT: {'first_int': 186102, 'second_int': 384426}
570528
计算 123*234+345*456+567*678 的结果是 570528。


此外，LangChain内置了一些运行时，可以直接使用

<div class="alert alert-success">
③使用AgentExecutor来运行Agent：
</div>


In [177]:
from langchain.agents import AgentExecutor

# 创建代理执行器实例，传入代理、工具和 verbose 参数设置为 True
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [178]:
# 调用代理执行器的 invoke 方法，传入输入信息
agent_executor.invoke({"input": "how many letters in the word education?"})



> Entering new AgentExecutor chain...
Let's count the letters in the word "education":

- e
- d
- u
- c
- a
- t
- i
- o
- n

That is 9 letters.

So, there are 9 letters in "education".

> Finished chain.


{'input': 'how many letters in the word education?',
 'output': 'Let\'s count the letters in the word "education":\n\n- e\n- d\n- u\n- c\n- a\n- t\n- i\n- o\n- n\n\nThat is 9 letters.\n\nSo, there are 9 letters in "education".'}

<div class="alert alert-success">
有没有什么问题？

可能某些版本不能成功调用到自定义的函数，原因可能是大家使用的某些版本存在一定bug，建议更新LangChain至最新版本。实际上，也可以将runtime和agentexecutor合在一起使用，具体见以下版本。大家可以体会一下两种方式有什么不同。
</div>

In [179]:
from langchain.agents import initialize_agent, AgentType, AgentExecutor  
from langchain.chat_models import ChatOpenAI  
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder  
from langchain.tools import tool   

"""定义提示"""
prompt_template = ChatPromptTemplate.from_messages([  
    ("system", "You are a powerful assistant. But you do not know how to calculate the length of a word"),  
    ("user", "{input}"), 
])  

"""定义模型"""
llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0.0)  

"""定义工具"""
@tool  
def get_word_length(word):  
    """获取单词的长度"""
    return len(word)  

tools = [get_word_length]  

"""定义代理"""
#use initialize_agent can "bind" the runtime and agentexecutor at the same time
#initialize_agent() return an agentexecutor object.
agent = initialize_agent(  
    tools,  
    llm,  
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, #use class AgentType() 
    verbose=True  
)  
"""运行代理"""
agent.run("how many letters in the word eduction?")



> Entering new AgentExecutor chain...


/tmp/ipykernel_23277/1042327408.py:26: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(
/tmp/ipykernel_23277/1042327408.py:33: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("how many letters in the word eduction?")


I need to find the length of the word "eduction".
Action: get_word_length
Action Input: "eduction"
Observation: 8
Thought:I now know the final answer
Final Answer: 8

> Finished chain.


'8'

**Memory**



大多数 LLM 应用程序都有一个对话界面。对话的一个重要组成部分是能够引用对话中较早介绍的信息。 至少，对话系统应该能够直接访问过去消息的某些窗口。 一个更复杂的系统需要有一个不断更新的世界模型，这允许它做一些事情，比如维护有关实体及其关系的信息。

我们将这种存储有关过去交互信息的能力称为“记忆”。

Memory在Langchain中的作用如下图所示。

<img src="images/memory.png" alt="langchainmemory" width="60%"></img>

--https://python.langchain.com/docs/modules/memory/

记忆系统需要支持两个基本动作：读取和写入。每条链都定义了一些执行逻辑，这些逻辑需要某些输入。 其中一些输入直接来自用户，但其中一些输入可能来自内存。 在给定的运行中，一条链将与其内存系统交互两次，分别是：

- 在接收到初始用户输入之后，但在执行核心逻辑之前，链将从其内存系统中读取并增加用户输入。
- 在执行核心逻辑之后，但在返回答案之前，链会将当前运行的输入和输出写入内存，以便在将来的运行中引用它们。

下面用一个简单的示例来展示Memory的使用

使用手动添加的方式将信息加入到内存，并记录为'history'

In [180]:
!pip show langchain

Name: langchain
Version: 0.3.13
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /home/ubuntu/anaconda3/envs/notebook@6.5.4/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community, langchain-experimental, paper-qa, swarms


In [181]:
from langchain.memory import ConversationBufferMemory

# 创建对话缓存记忆实例
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# 向对话记忆中添加用户消息
memory.chat_memory.add_user_message("hi!")
# 向对话记忆中添加AI消息
memory.chat_memory.add_ai_message("what's up?")

/tmp/ipykernel_23277/4222915812.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [182]:
# 加载记忆变量
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="what's up?", additional_kwargs={}, response_metadata={})]}

也可以对记忆进行命名

In [183]:
# 创建指定键值的对话缓存记忆实例
memory = ConversationBufferMemory(memory_key="chat_history")
# 向对话记忆中添加用户消息
memory.chat_memory.add_user_message("hi!")
# 向对话记忆中添加AI消息
memory.chat_memory.add_ai_message("what's up?")

In [184]:
memory.load_memory_variables({})

{'chat_history': "Human: hi!\nAI: what's up?"}

##### 端到端示例

In [185]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


# 初始化ChatOpenAI模型
llm = ChatOpenAI()
# 创建对话提示模板
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # 这里的`variable_name`必须与记忆中的对应
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# 注意我们设置`return_messages=True`以适配MessagesPlaceholder
# 注意`"chat_history"`与MessagesPlaceholder名称对应
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# 创建LLMChain实例，用于处理对话
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

/tmp/ipykernel_23277/2721332143.py:29: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(


现在可以进行多轮对话了

In [186]:
# 注意我们只传入`question`变量 - `chat_history`由记忆填充
conversation({"question": "hi"})
conversation({"question":"can you tell me a joke about apple and doctor?"})
conversation({"question":"That's funny! please tell me another"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi


/tmp/ipykernel_23277/515112740.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  conversation({"question": "hi"})



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you today?
Human: can you tell me a joke about apple and doctor?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you today?
Human: can you tell me a joke about apple and doctor?
AI: Sure! Here you go: 

Why did the apple go to the doctor?
Because it wasn't peeling well!
Human: That's funny! please tell me another

> Finished chain.


{'question': "That's funny! please tell me another",
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How are you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='can you tell me a joke about apple and doctor?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Sure! Here you go: \n\nWhy did the apple go to the doctor?\nBecause it wasn't peeling well!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="That's funny! please tell me another", additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm glad you liked it! Here's another one:\n\nWhy did the doctor carry a red pen?\nIn case they needed to draw blood!", additional_kwargs={}, response_metadata={})],
 'text': "I'm glad you liked it! Here's another one:\n\nWhy did the doctor carry a red pen?\nIn case they needed to draw blood!"}

**Callbacks**
LangChain提供了一个回调系统，允许您连接到LLM应用程序的各个阶段。这对于日志记录、监视、流式处理和其他任务非常有用。

您可以使用整个 API 中可用的参数来订阅这些事件。此参数是处理程序对象的列表，这些对象应实现下面更详细地描述的一个或多个方法。

以下列出了LangChain定义的基本Callbacks的Handler，Callbacks的使用有点类似“事件”。

这段代码仅用于展示，不可运行
```
class BaseCallbackHandler:
    """Base callback handler that can be used to handle callbacks from langchain."""

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        """Run when LLM starts running."""

    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs: Any
    ) -> Any:
        """Run when Chat Model starts running."""

    def on_llm_new_token(self, token: str, **kwargs: Any) -> Any:
        """Run on new LLM token. Only available when streaming is enabled."""

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""

    def on_llm_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        """Run when LLM errors."""

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> Any:
        """Run when chain starts running."""

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs: Any) -> Any:
        """Run when chain ends running."""

    def on_chain_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        """Run when chain errors."""

    def on_tool_start(
        self, serialized: Dict[str, Any], input_str: str, **kwargs: Any
    ) -> Any:
        """Run when tool starts running."""

    def on_tool_end(self, output: str, **kwargs: Any) -> Any:
        """Run when tool ends running."""

    def on_tool_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        """Run when tool errors."""

    def on_text(self, text: str, **kwargs: Any) -> Any:
        """Run on arbitrary text."""

    def on_agent_action(self, action: AgentAction, **kwargs: Any) -> Any:
        """Run on agent action."""

    def on_agent_finish(self, finish: AgentFinish, **kwargs: Any) -> Any:
        """Run on agent end."""
```

一个简单示例，演示Callbacks的使用方法

In [187]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# 初始化回调函数处理器
handler = StdOutCallbackHandler()
# 初始化ChatOpenAI模型
llm = ChatOpenAI(
    model="qwen-max",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0.0
)  
# 创建提示模板
prompt = PromptTemplate.from_template("1 + {number} = ")

# 没有构造回调函数：在初始化链时不设置StdOutCallbackHandler
print("不使用回调函数")
chain=LLMChain(llm=llm, prompt=prompt)
chain.invoke({"number":2})


不使用回调函数


{'number': 2, 'text': '1 + 2 = 3'}

In [188]:
print("\n使用回调函数的方式1")
# 构造回调函数：首先，在初始化链时明确设置StdOutCallbackHandler
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
chain.invoke({"number":2})


使用回调函数的方式1


> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


{'number': 2, 'text': '1 + 2 = 3'}

In [189]:
print("\n使用回调函数的方式2")
# 使用verbose标志：然后，让我们使用`verbose`标志来实现相同的结果
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
chain.invoke({"number":2})


使用回调函数的方式2


> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


{'number': 2, 'text': '1 + 2 = 3'}

In [190]:
print("\n使用回调函数的方式3")
# 请求回调函数：最后，让我们使用请求中的`callbacks`来实现相同的结果
chain = LLMChain(llm=llm, prompt=prompt)
chain.invoke({"number":2}, {"callbacks":[handler]})


使用回调函数的方式3


> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


{'number': 2, 'text': '1 + 2 = 3'}

<div class="alert alert-warning">
除了以上列出了LangChain定义的回调函数外，也支持自定义回调函数。
    
注意：不要把Fallbacks和Callbacks混淆
</div>

## 三、LangChain的使用场景

下面介绍2个LangChain的典型应用场景，具体实现，将在下节课进行。

### 聊天机器人
<img src="images/chatbot.png" alt="Chatbots" width="60%"></img>

--https://python.langchain.com/docs/use_cases/chatbots

聊天机器人是LLM最常见的应用，除了LLM的提示回复功能外，还应该具有记忆和检索的功能。记忆是指能记住过去的交互情况，检索则是要能够向机器人提供最新的或特定领域的信息。

聊天模型界面基于消息而不是原始文本。聊天需要考虑以下几个重要因素：

- chat model：即一个基础的LLM
- prompt template：用于将默认消息、用户输入、聊天历史记录和其他检索上下文的提示等内空组合成一个prompt
- memory：记忆功能
- Retriever：检索系统，这部分对于构建具有专业能力的机器人十分重要

### 基于结构化数据的问答
<img src="images/sqlagent.png" alt="QA over structed data" width="60%"></img>

--https://python.langchain.com/docs/use_cases/qa_structured/sql

可以使用LangChain提供的工具来与SQL数据库进行交互

- Build SQL queries：基于用户的自然语言提问
- Query a SQL database：使用链来创建和执行查询语句
- Interact with a SQL database：使用agent实现强大灵活的查询

此外，可支持的应用还有信息抽取、长文本总结、文本标注、网页抓取等，也可以与各类图数据库，如Neo4j等构建基于图数据的问答链。

## 环境配置
<div class="alert alert-info">

1. **langchain-core:** <br>
    作用：包含已作为标准出现的简单核心抽象。这些抽象的示例包括语言模型、文档加载器、嵌入模型、向量存储、检索器等的抽象。
2. **langchain_openaity:** <br>
    作用：包含LangChain与openai通信需要的库。
3. **langchain:** <br>
    作用：包含更高级别和特定于用例的链、代理和检索算法，它们是应用程序认知架构的核心。
4. **pypdf:** <br>
    作用：用于读取 PDF 文件中的文本内容。
5. **python-dotenv** <br>
    作用：用于导入环境变量，如GPT-key等。
6. **semantic-kernel==0.3.15.dev0:** <br>
    作用：semantic-kernel的核心库。
7. **pydantic==1.10.2:** <br>
    作用：用于进行数据验证，与semantic-kernel配合使用，需要配合semantic-kernel的版本。
8. **openai==0.28:** <br>
    作用：包含使用python与openai通信需要的库，需要配合semantic-kernel的版本。